In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import urllib, json
import MartianWeather as mw
from PIL import Image
import datetime

In [33]:
f = open('API_KEY.txt', 'r')
api_key = f.read()
f.close()



GET https://api.nasa.gov/neo/rest/v1/feed?start_date=START_DATE&end_date=END_DATE&api_key=API_KEY

In [34]:
#def get_url():
#    url_level_0 = urllib.request.urlopen(url)
#    data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
#    return 

In [35]:
# Define API key
api_root = 'https://api.nasa.gov/'
api_product = 'neo/rest/v1/feed?'
start_date = '2020-07-07'
end_date = '2020-07-07'
url = f"{api_root}{api_product}start_date={start_date}&end_date={end_date}&api_key={api_key}"

In [36]:
url_level_0 = urllib.request.urlopen(url)
data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
data_level_0

,links,element_count,near_earth_objects
next,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
prev,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
self,http://api.nasa.gov/neo/rest/v1/feed?start_dat...,11,NaN
2020-07-07,NaN,11,[{'links': {'self': 'http://api.nasa.gov/neo/r...


In [44]:
data_day = data_level_0.near_earth_objects[3]
data_day_item = data_day[0]
data_day_item

{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/3343102?api_key=8Mbg6rxSagXNYxrUbC8ynidJUcLn9h4CC7ucuT21'},
 'id': '3343102',
 'neo_reference_id': '3343102',
 'name': '(2006 SS19)',
 'nasa_jpl_url': 'http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3343102',
 'absolute_magnitude_h': 18.9,
 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.4411182,
   'estimated_diameter_max': 0.9863702813},
  'meters': {'estimated_diameter_min': 441.1181999969,
   'estimated_diameter_max': 986.3702813054},
  'miles': {'estimated_diameter_min': 0.2740980571,
   'estimated_diameter_max': 0.6129018881},
  'feet': {'estimated_diameter_min': 1447.2382352778,
   'estimated_diameter_max': 3236.1230737181}},
 'is_potentially_hazardous_asteroid': False,
 'close_approach_data': [{'close_approach_date': '2020-07-07',
   'close_approach_date_full': '2020-Jul-07 08:13',
   'epoch_date_close_approach': 1594109580000,
   'relative_velocity': {'kilometers_per_second': '11.947301649',
    'kilometers_per_h

In [42]:
data_day_item['absolute_magnitude_h']

18.9